In [1]:
import numpy as np
import napari
from skimage.measure import label
import cc3d
from tifffile import imread, imwrite
from glob import glob
from utils import *
from skimage.morphology import disk, binary_dilation
import matplotlib.pyplot as plt
import pandas

Project uses napari viewer for interaction with images. Installation of napari is necessary. 

In [2]:
# start the viewer
viewer = napari.Viewer()

In [3]:
# if raw images are tiff filees
path = 'hek ctrl J F/HEK J F 5a.ome.tif'
raw = imread(path)

viewer.add_image(raw, name='raw')

<Image layer 'raw' at 0x29b412fa0>

Load respective masks for the raw images. Masks should be time-stacks saved as tiff files.

In [4]:
# open segmentation
path = 'hek ctrl J F/HEK J F 5a_cells.ome.tif'
cells = imread(path)

path = 'hek ctrl J F/HEK J F 5a_sperm.ome.tif'
sperm = imread(path)

# label masks
sperm_labels = label(sperm)
cell_labels = label(cells)

# add both labels to napari viewer
viewer.add_labels(cell_labels, name='cells')
viewer.add_labels(sperm_labels, name='sperm')

<Labels layer 'sperm' at 0x2afb08a90>

To select which cell to analyze open viewer and add "points" layer, then add points over all sperm cells of interest.

In [6]:
# choose cell
points = viewer.layers['Points'].data

for point in points:
    cells = viewer.layers['cells'].data
    sperm = viewer.layers['sperm'].data

    # take cells value from point layer
    sperm_id = sperm[0][int(point[1])][int(point[2])]

    # take only cell with cell_id
    sperm_cell = sperm.copy()
    sperm_cell[sperm != sperm_id] = 0

    # dilate cell
    sperm_cell_dilated = np.zeros_like(sperm_cell)

    for i in range(sperm_cell.shape[0]):
        # dilate using cv2
        kernel = np.ones((3, 3), np.uint8)
        sperm_cell_dilated[i] = cv2.dilate(sperm_cell[i].astype(np.uint8), kernel=kernel, iterations=8)

    # if any sperm is touching cell dilated, then sperm is interacting with cell
    cell = viewer.layers['cells'].data
    interacting_labels = np.multiply(sperm_cell_dilated, cell)

    sperm_interacting_id, counts = np.unique(interacting_labels, return_counts=True)
    sperm_interacting_id = sperm_interacting_id[counts > 1000]
    sperm_interacting_id = sperm_interacting_id[1:]

    for k, interaction in enumerate(sperm_interacting_id):
        print('iteration: ', k)
        # if sperm and sperm interacting overlap, get this sperm value
        # sperm_cell = viewer.layers['sperm_cell'].data

        print('sperm_cell shape is', sperm_cell.shape, 'with labels', np.unique(sperm_cell))
        
        # create an empty array for angles values
        angles = []

        # do angle measurements for all time frames
        
        # find stack where the overlap is present
        overlap = interacting_labels.copy()
        overlap[overlap != interaction] = 0

        for stack in overlap:
            if np.sum(overlap) > 0:
                # get overlap coordinates
                overlap_coords = np.array(np.where(stack != 0))
                break

        # compute cell line and cell vector
        cell_line, cell_vector = compute_cell_vector(overlap_coords)

        # print('cell_line', cell_line)

        # n = 2 * sperm_cell.shape[0]
        # d = 3
        # pos = np.zeros(shape=(n, 2, d), dtype=np.float32)
        # vector_number = 0

        for i, sperm in enumerate(sperm_cell):
            sperm_line, sperm_vector = compute_sperm_cell_vector(sperm)
            intersection = find_intersection(cell_line, sperm_line)

            if intersection == None:
                continue

            angle = compute_angle(cell_vector, sperm_vector)
            print(f'angle between cell and sperm on plane {i}: ', angle)
            angles.append(angle)

            # if i == 1:
            # pos[vector_number, 0, :] = (i, *intersection)
            # pos[vector_number, 1, :] = (i, *sperm_vector)
            # vector_number += 1

            # pos[vector_number, 0, :] = (i, *intersection)
            # pos[vector_number, 1, :] = (i, *cell_vector)
            # vector_number += 1
            
            
        # vect = viewer.add_vectors(pos, edge_width=1, length=10, name='cell vect')

        # create dataframe
        df = pandas.DataFrame(angles)
        # rename column
        df.columns = ['angle']
        # add new column with same value
        df['time'] = np.arange(len(angles))
        # compute angle change
        df['angle_change'] = abs(df['angle'].diff())
        # save to csv
        df.to_csv(f'cell_{point[0][1], point[0][2]}_{k}.csv')
        print(df)

: 

: 